### Emoji Frequencies

In [1]:
import pandas as pd
import numpy as np
import re
from dask.distributed import Client
import dask.bag as db
import dask.dataframe as dd
import json

import emoji

In [2]:
from dask.distributed import Client, LocalCluster

# set up cluster and workers
cluster = LocalCluster(n_workers=4, 
                       threads_per_worker=1,
                       memory_limit='32GB')
client = Client(cluster)

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 119.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51854,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 119.21 GiB
Comm: tcp://127.0.0.1:51864,Total threads: 1
Dashboard: http://127.0.0.1:51866/status,Memory: 29.80 GiB
Nanny: tcp://127.0.0.1:51856,


In [ ]:
cluster.close()
client.close()

distributed.utils_perf - WARNING - full garbage collections took 19% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument
distributed.comm.tcp - WARNING - Could not set timeout on TCP stream: [Errno 22] Invalid argument


In [5]:
del df

### Turkish

In [12]:
path_parquet = "/Users/atanaltay/Documents/Research/inwork/marketing_emotions/codes/data/all_marketing_data/tr_emoanalysis.parquet"

df = dd.read_parquet(path_parquet, engine='pyarrow',columns=['text'])
bg = db.from_sequence(df.text)


## create new emoji counts - compound emojis are counted as one
to achieve this all emojis are converted to textual form with their short names



In [14]:
result = bg.map(lambda line: emoji.demojize(line)).map(lambda demojized: re.findall(r'(:[!_\-\w]+:)', demojized)).flatten()\
    .map(lambda item: item.replace(":",'')).frequencies().compute()

In [22]:
df_count = pd.DataFrame(result,columns=["key","cnt"])
df_count.index = "_"+df_count["key"]+"_"
df_count.drop(columns='key',inplace=True)
df_count.to_csv('/Users/atanaltay/Documents/Research/inwork/marketing_emotions/codes/data/all_marketing_data/emoticon_analysis/tr_emoticon_counts.csv')

In [26]:
del df
del df_count
del bg

In [35]:
client.close()


### English

In [4]:

path_parquet = "/Users/atanaltay/Documents/Research/inwork/marketing_emotions/codes/data/all_marketing_data/en_emoanalysis.parquet"

df = dd.read_parquet(path_parquet, engine='pyarrow',columns=['text'])
bg = db.from_sequence(df.text)





In [5]:
result = bg.map(lambda line: emoji.demojize(line)).map(lambda demojized: re.findall(r'(:[!_\-\w]+:)', demojized)).flatten()\
    .map(lambda item: item.replace(":",'')).frequencies().compute()




distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


In [6]:
df_count = pd.DataFrame(result,columns=["key","cnt"])
df_count.index = "_"+df_count["key"]+"_"
df_count.drop(columns='key',inplace=True)
df_count.to_csv('/Users/atanaltay/Documents/Research/inwork/marketing_emotions/codes/data/all_marketing_data/emoticon_analysis/en_emoticon_counts.csv')

### English with plain text reading

In [8]:
import jsonlines
#writer = jsonlines.open("data/all_marketing_data/en_test.json", mode='a')

emoji_count = dict()

with jsonlines.open("/Users/atanaltay/Documents/Research/inwork/marketing_emotions/codes/data/all_marketing_data/en_for_emo_merged_sept.json") as reader:
    
    cnt = 8101034
    iteration = 0
    for line in reader:
        str_demojized =  emoji.demojize(line['text'])
        emojis_list= re.findall(r'(:[!_\-\w]+:)', str_demojized)
        for item in emojis_list:
            if item in emoji_count:
                     emoji_count[item]+=1
            else:
                emoji_count[item]=1
        iteration+=1
        
        if(iteration%1000==0):
            print("completed:",iteration,"/",cnt, end="\r")

df_count = pd.DataFrame(emoji_count.items(),columns=["key","count"])
df_count.key = df_count.key.str.replace(':','_')
df_count.index = df_count.key
df_count.drop(columns='key',inplace=True)
df_count.to_csv('/Users/atanaltay/Documents/Research/inwork/marketing_emotions/codes/data/all_marketing_data/emoticon_analysis/en_emoticon_counts.csv')
print("completed!....")

completed!....1000 / 8101034


In [3]:
!wc -l /Users/atanaltay/Documents/Research/inwork/marketing_emotions/codes/data/all_marketing_data/en_for_emo_merged_sept.json

 8101034 /Users/atanaltay/Documents/Research/inwork/marketing_emotions/codes/data/all_marketing_data/en_for_emo_merged_sept.json
